In [27]:
%matplotlib inline

import os

import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.preprocessing.image import ImageDataGenerator

random_state = 42

In [30]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
y_train = np.eye(10)[y_train.astype('int32').flatten()]

x_test = x_test.astype('float32') / 255
y_test = np.eye(10)[y_test.astype('int32').flatten()]

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=10000)

In [34]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', kernel_initializer='he_normal', input_shape=(32, 32, 3)))  # 32x32x3 -> 28x28x6
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x6 -> 14x14x6
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', kernel_initializer='he_normal'))  # 14x14x6 -> 10x10x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 10x10x16 -> 5x5x16

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu', kernel_initializer='he_normal'))  # 400 ->120
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  # 120 ->84
model.add(Dense(10, activation='softmax'))  # 84 ->10

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)


In [35]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,  # 3.1.1 左右にずらす
    height_shift_range=0.2,  # 3.1.2 上下にずらす
    horizontal_flip=True,  # 3.1.3 左右反転
    # 3.2.1 Global Contrast Normalization (GCN) (Falseに設定しているのでここでは使用していない)
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False # 3.2.2 Zero-phase Component Analysis (ZCA) Whitening (Falseに設定しているのでここでは使用していない)
)

In [36]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    steps_per_epoch=x_train.shape[0] // 100, epochs=30, validation_data=(x_valid, y_valid))

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


400/400 [==============================] - 19s 43ms/step - loss: 1.8962 - accuracy: 0.3013 - val_loss: 1.6436 - val_accuracy: 0.4074
Epoch 2/30
400/400 [==============================] - 17s 42ms/step - loss: 1.6727 - accuracy: 0.3841 - val_loss: 1.4610 - val_accuracy: 0.4627
Epoch 3/30
400/400 [==============================] - 17s 43ms/step - loss: 1.5742 - accuracy: 0.4281 - val_loss: 1.4359 - val_accuracy: 0.4821
Epoch 4/30
400/400 [==============================] - 16s 40ms/step - loss: 1.5307 - accuracy: 0.4459 - val_loss: 1.3221 - val_accuracy: 0.5291
Epoch 5/30
400/400 [==============================] - 17s 42ms/step - loss: 1.4780 - accuracy: 0.4664 - val_loss: 1.3634 - val_accuracy: 0.5148
Epoch 6/30
400/400 [==============================] - 16s 41ms/step - loss: 1.4436 - accuracy: 0.4804 - val_loss: 1.3140 - val_accuracy: 0.5291
Epoch 7/30
400/400 [==============================] - 16s 41ms/step - loss: 1.4205 - accuracy: 0.4902 - val_loss: 1.3552 - val_accuracy: 0.5182
Epo

In [38]:
model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 1s 3ms/step - loss: 1.1098 - accuracy: 0.6073


[1.109781265258789, 0.6072999835014343]